<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/Atividade1-Ciclo2-RESPOSTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Exercício 1 - Dataset Digits do sklearn

# Acesso: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

# 1) Importar o pacote "sklearn.datasets" e o "load_digits"
# 2) Carregar o dataset através do método: load_digits()
# 3) Observe as keys do dataset usando o método "keys"
# 4) A chave "data" são as features e a chave "target" é o y. Separe os dados em 2 variáveis diferentes
# 5) Separe o conjunto de dados em treinamento e teste usando o método: "train_test_split"

# 6) Treinar o K-Means, sendo o K seja igual ao número de classes do dataset
 #(Lembre-se que o predict do K-Means irá retornar o cluster que o dado pertence mas não necessariamente esse número do cluster é a classe correta. Não esqueça de fazer o mapeamento do "cluster x classe",
 # conforme feito no link: https://colab.research.google.com/github/tmoura/machinelearning/blob/master/KMeans.ipynb)
# 7) Mostrar a taxa de acerto do K-Means
# 8) Calcular a curva do cotovelo para verificar qual o número ideal de clusters para esse dataset
# 9) Calcular o Coeficiente de Silhouette para o K-Means treinado em (7)
# 10) É possível melhorar o Coeficiente de Silhouette? Altere o K-Means para conseguir um valor melhor do Coeficiente de Silhouette.

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

df = load_digits()

y = df.target
X = df.data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y) # 80% treino e 20% teste

myset = set(y_train) # Cria um conjunto. Em conjuntos, dados não se repetem. Assim, esse conjunto conterá apenas um valor de cada, ou seja: [1,2,3]
clusters = len(myset) # Quantos clusters teremos no KMeans

model = KMeans(n_clusters = clusters)
model = model.fit(X_train)

In [ ]:
from collections import Counter

# Pegar os labels dos padrões de Treinamento
labels = model.labels_

map_labels = []

for i in range(clusters):
  map_labels.append([])

new_y_train = y_train.tolist()

for i in range(len(y_train)):
  for c in range(clusters):
    if labels[i] == c:
      map_labels[c].append(new_y_train[i])

#print(map_labels)

# Criar dicionário com os labells a serem mapeados
mapping = {}

for i in range(clusters):
  final = Counter(map_labels[i]) # contar a classe que mais aparece
  value = final.most_common(1)[0][0] # retorna a classe com maior frequência
  mapping[i] = value

In [ ]:
import matplotlib.pyplot as plt
sse = {}
for k in range(1, 30): # Inicialmente teríamos 10 clusters
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X)
    sse[k] = kmeans.inertia_ # Inertia: soma das distâncias das instâncias ao centroide mais próximo. Representa o quão coerentes são os clusters internamente.
    print(sse[k])

plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Número de clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
from sklearn import metrics

result = model.predict(X_test)
result = [mapping[i] for i in result]

acc = metrics.accuracy_score(result, y_test)
show = round(acc * 100)
print("{}%".format(show))

print("Índice Silhouette: %0.3f" % metrics.silhouette_score(X,y)) #

In [ ]:
## Exercício 2 - Dataset Wine do sklearn

# Acesso: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine

# 1) Importar o pacote "sklearn.datasets" e o "load_wine"
# 2) Carregar o dataset através do método: load_wine()
# 3) Observe as keys do dataset usando o método "keys"
# 4) A chave "data" são as features e a chave "target" é o y. Separe os dados em 2 variáveis diferentes
# 5) Usar todo o conjunto de dados para treinamento
# 6) Treinar o DBScan. Quantos clusters foram gerados com os parâmetros que vc escolheu?
# 7) Calcular o Coeficiente de Silhouette para o DBScan treinado em (6)
# 8) É possível melhorar o Coeficiente de Silhouette? Altere os parâmetros do DBScan para conseguir um valor melhor do coeficiente

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn import preprocessing
import pandas as pd

df = load_wine(as_frame=True)
df = df.frame

y = df['target']
df.drop('target',axis='columns', inplace=True)

X = df

x2 = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x2)
X = pd.DataFrame(x_scaled)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y) # 80% treino e 20% teste

myset = set(y_train) # Cria um conjunto. Em conjuntos, dados não se repetem. Assim, esse conjunto conterá apenas um valor de cada, ou seja: [1,2,3]
clusters = len(myset) # Quantos clusters teremos no KMeans

# Instância do DBSCAN e aprendizado
model = DBSCAN(eps=0.5, min_samples=5).fit(X)
core_samples_mask = np.zeros_like(model.labels_, dtype=bool) # Cria matriz de valores False
core_samples_mask[model.core_sample_indices_] = True # db.core_sample_indices_ são os índices dos dados que pretencem a algum cluster
labels = model.labels_ # Clusters ao qual pertencem os dados

display(labels)

In [ ]:
# Número de clusters
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Número estimado de clusters: %d" % n_clusters_)
print("Número estimado de outliers ou ruídos: %d" % n_noise_) # dados que não pertencem a nunhum cluster
print("Índice Silhouette: %0.3f" % metrics.silhouette_score(X, y)) # retorna a média do índice de silhouette para todos os exemplos